In [ ]:
# 7-1
!mkdir chap7
%cd ./chap7

In [2]:
# 7-3
import random
import glob
import json
from tqdm import tqdm

import torch
from torch.utils.data import DataLoader
from transformers import BertJapaneseTokenizer, BertModel
import pytorch_lightning as pl

# 日本語の事前学習モデル
MODEL_NAME = "tohoku-nlp/bert-base-japanese-whole-word-masking"

In [3]:
# 7-4（説明コメント付き：BERTを用いたマルチラベル文分類の最小実装）
# --------------------------------------------------------------------------------
# 目的：
#  - 事前学習済み BERT（エンコーダ本体：BertModel）の出力（最終層隠れ状態）から
#    「[PAD] を除いたトークン平均（mean pooling）」で文ベクトルを作り、
#    線形層でラベル数（num_labels）次元に写像して **マルチラベル**分類を行う。
#
# 理論メモ（マルチラベル vs マルチクラス）：
#  - マルチクラス（排他的1クラス）では softmax + CrossEntropyLoss を用いるが、
#  - マルチラベル（複数ラベル同時に1の可能性）では **各ラベルを独立なベルヌーイ**として扱い、
#    出力は raw logits（活性前）→ **BCEWithLogitsLoss**（内部で sigmoid + BCE を数値安定に計算）を用いる。
#  - 教師ラベルの形：shape [B, C]、各要素は {0,1}（float型推奨）。しきい値 0.5 などで陽性判定を後段で行う。
#
# プーリング設計（mean pooling の意図）：
#  - [CLS] ベクトル単独の利用に比べ、文全体の情報を平均で取り入れやすい。短文～中程度の長さで堅実なベースライン。
#  - attention_mask により [PAD] 位置は平均から除外（分母は非PADトークン数）。
#  - 分母ゼロ（全PAD）対策：学習データ生成で通常発生しない想定だが、実務では clamp などで保険をかけると安全。
#
# そのほか設計の注意：
#  - token_type_ids（= segment ids）は文対（sentence pair）で区別に使うが、単文では 0 固定でもよい。
#  - 出力は慣例的には transformers の `SequenceClassifierOutput` を使うが、
#    ここでは attributes にアクセスできる簡易オブジェクトを返す（互換性が必要なら差し替え推奨）。
#  - クラス不均衡が強い場合は BCEWithLogitsLoss の `pos_weight` を利用すると勾配が安定する。
# --------------------------------------------------------------------------------


class BertForSequenceClassificationMultiLabel(torch.nn.Module):

    def __init__(self, model_name, num_labels):
        super().__init__()
        # BertModel のロード（分類ヘッドなしの本体のみ）。
        # - 入力：input_ids, attention_mask, token_type_ids
        # - 出力：last_hidden_state（[B, L, H]）など
        self.bert = BertModel.from_pretrained(model_name)

        # 文ベクトル（プーリング後の [B, H]）から各ラベルのロジット [B, C] を直線写像する層。
        # - H: 隠れ次元（bert.config.hidden_size）、C: ラベル数（num_labels）
        self.linear = torch.nn.Linear(self.bert.config.hidden_size, num_labels)

    def forward(
        self, input_ids=None, attention_mask=None, token_type_ids=None, labels=None
    ):
        # 1) BERT でトークン列を符号化し、最終層の隠れ状態を得る。
        #    last_hidden_state の形は [B, L, H]（B:バッチ、L:系列長、H:隠れ次元）
        bert_output = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
        )
        last_hidden_state = bert_output.last_hidden_state

        # 2) [PAD] 以外の位置のみで平均プーリングして文ベクトルを作る。
        #    - attention_mask は [B, L] の {0,1}。unsqueeze(-1) で [B, L, 1] に拡張し、
        #      last_hidden_state（float）にマスク乗算→ PAD 位置を 0 化。
        #    - 分母は非PADトークン数（attention_mask.sum(dim=1)）。
        #      典型データでは 0 になることはないが、実運用では clamp_min(1e-9) などで安全策も検討。
        averaged_hidden_state = (last_hidden_state * attention_mask.unsqueeze(-1)).sum(
            1
        ) / attention_mask.sum(1, keepdim=True)
        # 参考（安全策の例、実装切替の際に利用）：
        # denom = attention_mask.sum(1, keepdim=True).clamp_min(1e-9).float()
        # masked = last_hidden_state * attention_mask.unsqueeze(-1).float()
        # averaged_hidden_state = masked.sum(1) / denom

        # 3) 線形層でラベル数 C 次元のロジットへ（活性はかけない：BCEWithLogitsLoss を使うため）
        scores = self.linear(averaged_hidden_state)  # shape: [B, C]

        # 4) 出力を dict 形式で用意（logits を必須、labels が来ていれば loss も計算）
        output = {"logits": scores}

        # 5) 教師ラベルが与えられた場合は損失を計算して返す。
        #    - BCEWithLogitsLoss は内部で sigmoid を組み合わせた数値安定版。
        #    - labels 形状は [B, C]、型は float（{0.0, 1.0}）を想定。
        #    - クラス不均衡が強ければ pos_weight（shape [C]）の指定を検討。
        if labels is not None:
            loss = torch.nn.BCEWithLogitsLoss()(scores, labels.float())
            output["loss"] = loss

        # 6) 呼び出し側が `output.logits` / `output.loss` で参照できるよう簡易オブジェクト化。
        #    transformers の標準 `SequenceClassifierOutput` を使う場合は差し替え可。
        output = type("bert_output", (object,), output)

        return output

In [4]:
# 7-5（説明コメント付き：マルチラベル文分類モデルのロードとデバイス配置）
# -----------------------------------------------------------------------------
# 目的：
#  - 日本語BERTと同一語彙のトークナイザをロードし、
#  - 7-4で定義した「BertForSequenceClassificationMultiLabel」を初期化、
#  - 環境に応じた最適デバイス（MPS/CUDA/CPU）へ安全に配置する。
#
# 理論メモ：
#  - 本モデルは出力が各ラベルの「ロジット」（活性前）で、マルチラベル想定（独立ベルヌーイ）。
#    学習時は BCEWithLogitsLoss（sigmoid + BCE の安定版）を用いる。
#  - num_labels=2 の場合、2つのラベルそれぞれを {0,1} で同時に予測する（softmaxではない）。
# -----------------------------------------------------------------------------

import torch
from transformers import BertJapaneseTokenizer

# --- MODEL_NAME が未定義でも動くように保険を入れる ---
try:
    MODEL_NAME
except NameError:
    MODEL_NAME = (
        "tohoku-nlp/bert-base-japanese-whole-word-masking"  # 東北大版日本語BERT（WWM）
    )


# --- デバイス選択：Mac(MPS) → CUDA → CPU の順で自動選択 ---
def get_best_device() -> torch.device:
    # Apple Silicon + macOS Metal
    if torch.backends.mps.is_available() and torch.backends.mps.is_built():
        return torch.device("mps")
    # NVIDIA GPU（Linux/Windowsなど）
    if torch.cuda.is_available():
        return torch.device("cuda")
    # フォールバック：CPU
    return torch.device("cpu")


device = get_best_device()
print(f"[info] using device = {device}")

# --- トークナイザのロード（モデルと同じ語彙でないとIDがずれて壊れる） ---
tokenizer = BertJapaneseTokenizer.from_pretrained(MODEL_NAME)

# --- マルチラベル分類モデルの構築 ---
# num_labels=2：2つのラベルを独立に予測（例：{スポーツ, IT} の属否を同時に判定 など）
bert_scml = BertForSequenceClassificationMultiLabel(MODEL_NAME, num_labels=2)

# --- 最適デバイスへ配置（.cuda()固定はMacで失敗するため .to(device) を統一使用） ---
bert_scml = bert_scml.to(device)

# 推論時は Dropout を止めたいので eval()、学習開始時は train() を呼ぶ。
# bert_scml.eval()   # 例：推論前
# bert_scml.train()  # 例：学習ループ開始時

# （参考）出力の扱い：
# - forward は {'logits': Tensor[B, C], 'loss': Tensor[]} 相当を返す簡易オブジェクト。
# - 学習時は labels=[B, C] を与えると BCEWithLogitsLoss を内部計算し output.loss に入る。

[info] using device = mps


In [5]:
# 7-6（説明コメント付き：マルチラベル推論・しきい値判定・サブセット精度）
import torch

# 入力テキスト：各サンプルに複数ラベルが立ち得る（マルチラベル）
text_list = ["今日の仕事はうまくいったが、体調があまり良くない。", "昨日は楽しかった。"]

# 教師ラベル（shape=[B, C]）：
#  - マルチラベルなので各列（ラベル）を独立に {0,1} で表す
#  - 例：C=2 のとき [仕事/体調] のような2軸を同時に予測するイメージ
labels_list = [[1, 1], [0, 1]]

# --- デバイス整合：モデル(bert_scml)の実デバイスに入力を合わせる ---
device = next(bert_scml.parameters()).device

# データの符号化
#  - padding='longest' はバッチ内の最長系列に合わせて動的にPADを付与
#  - 実運用では max_length+truncation=True+padding='max_length' で固定長化すると計算が安定
encoding = tokenizer(text_list, padding="longest", return_tensors="pt")
encoding = {k: v.to(device) for k, v in encoding.items()}

# 教師ラベルテンソル
#  - 学習で BCEWithLogitsLoss を使う場合は float（{0.0,1.0}）が望ましい
#  - ここでは評価用に int 版も用意（比較演算時に便利）
labels_float = torch.tensor(labels_list, dtype=torch.float32, device=device)
labels_int = labels_float.to(torch.int64)

# --- 推論：本モデルは logits（活性前スコア）を返す ---
with torch.no_grad():
    output = bert_scml(**encoding)
scores = output.logits  # shape: [B, C]，各ラベルのロジット

# --- 判定（しきい値） ---
# 方式A：確率で判定（推奨。しきい値を柔軟に最適化できる）
probs = scores.sigmoid()  # p(y_c=1|x)
threshold = 0.5  # 必要に応じて ROC/PR で最適化
labels_predicted = (probs >= threshold).to(torch.int64)

# 方式B：ロジットの符号で判定（sigmoid≥0.5 と等価だが明示性に欠ける）
# labels_predicted = (scores > 0).to(torch.int64)

# --- 精度計算（subset accuracy：全ラベル一致率） ---
#  - サンプルごとに全てのラベルが正解しているかを判定し、その平均をとる厳しめの指標
num_correct = (labels_predicted == labels_int).all(dim=-1).sum().item()
accuracy = num_correct / labels_int.size(0)

# 参考：他の評価指標（マルチラベルで一般的）
#  - example-based F1（サンプル単位のF1を平均）
#  - micro/macro-F1（ラベル軸で集計）
#  - Hamming loss（ラベル単位の誤り率）
# これらは torchmetrics（MultilabelF1Score 等）で容易に算出可能

print("# scores (logits):", scores)
print("# probs (sigmoid):", probs)
print("# predicted labels:", labels_predicted)
print("# subset accuracy:", accuracy)

# scores (logits): tensor([[0.2937, 0.1129],
        [0.2349, 0.0266]], device='mps:0')
# probs (sigmoid): tensor([[0.5729, 0.5282],
        [0.5585, 0.5066]], device='mps:0')
# predicted labels: tensor([[1, 1],
        [1, 1]], device='mps:0')
# subset accuracy: 0.5


In [6]:
# 7-7（説明コメント付き：マルチラベル損失（BCEWithLogitsLoss）の計算）
# -----------------------------------------------------------------------------
# 目的：
#  - トークナイズ結果にマルチラベル教師（[B, C]）を同梱し、モデルの forward で
#    BCEWithLogitsLoss（sigmoid + BCE の数値安定版）を自動計算させて loss を得る。
#
# 理論メモ：
#  - マルチラベルでは各クラスを独立なベルヌーイとみなし、出力はロジット（活性前）。
#    教師は float 型の {0.0, 1.0} 行列（shape=[B, C]）。softmax は使わない。
#  - BCEWithLogitsLoss は内部で sigmoid を合成するため、出力に sigmoid をかけずに
#    生のロジットをそのまま渡すのが正しい。
#  - クラス不均衡が強い場合は pos_weight（shape=[C]）を指定すると陽性側の損失寄与を補正できる。
# -----------------------------------------------------------------------------

import torch

# --- デバイス整合：モデル(bert_scml)の実デバイスに入力を合わせる ---
device = next(bert_scml.parameters()).device

# --- データの符号化（バッチ内の最長に合わせて動的PAD） ---
# 実運用では max_length+truncation=True+padding="max_length" で固定長化すると計算が安定。
encoding = tokenizer(text_list, padding="longest", return_tensors="pt")
encoding = {k: v.to(device) for k, v in encoding.items()}

# --- ラベルを同梱（[B, C]、float 型） ---
# BCEWithLogitsLoss は float を前提。int のままでも内部で float 化されるが明示が安全。
labels = torch.tensor(labels_list, dtype=torch.float32, device=device)
encoding["labels"] = labels

# --- 損失の計算 ---
# 学習時は train() にして Dropout を有効、評価時は eval() + no_grad()。
bert_scml.train()  # 例：学習フェーズを想定（評価なら bert_scml.eval() と no_grad() を併用）
output = bert_scml(**encoding)
loss = output.loss  # BCEWithLogitsLoss（バッチ平均）

print("# loss:", float(loss.detach().cpu()))

# （参考）不均衡対策：pos_weight を使う例（実装差し替え時）
#   C = labels.size(1)
#   pos_weight = torch.tensor([...], dtype=torch.float32, device=device)  # shape=[C]
#   criterion = torch.nn.BCEWithLogitsLoss(pos_weight=pos_weight)
#   loss = criterion(output.logits, labels)

# loss: 0.671375036239624


In [8]:
# 7-9
data = json.load(open("chABSA-dataset/e00030_ann.json"))
print(data["sentences"][0])

{'sentence_id': 0, 'sentence': '当期におけるわが国経済は、景気は緩やかな回復基調が続き、設備投資の持ち直し等を背景に企業収益は改善しているものの、海外では、資源国等を中心に不透明な状況が続き、為替が急激に変動するなど、依然として先行きが見通せない状況で推移した', 'opinions': [{'target': 'わが国経済', 'category': 'OOD#general', 'polarity': 'neutral', 'from': 6, 'to': 11}, {'target': '景気', 'category': 'OOD#general', 'polarity': 'positive', 'from': 13, 'to': 15}, {'target': '設備投資', 'category': 'OOD#general', 'polarity': 'positive', 'from': 28, 'to': 32}, {'target': '企業収益', 'category': 'OOD#general', 'polarity': 'positive', 'from': 42, 'to': 46}, {'target': '資源国等', 'category': 'OOD#general', 'polarity': 'neutral', 'from': 62, 'to': 66}, {'target': '為替', 'category': 'OOD#general', 'polarity': 'negative', 'from': 80, 'to': 82}]}


In [9]:
# 7-10（説明コメント付き：chABSA からマルチラベル（negative/neutral/positive）データセットを構築）
# --------------------------------------------------------------------------------
# 目的：
#  - chABSA の JSON 群（各ファイルに sentences が入っている）から、
#    文テキストと極性ラベル（3値：negative/neutral/positive）を **マルチラベル** 形式で取り出す。
#    * ABSA は 1 文中に複数の「意見（aspect-opinion）」が存在し得るため、
#      文レベルでは negative/neutral/positive が**同時に立つ**可能性を想定（= マルチラベル）。
#
# 理論メモ：
#  - ここで作る labels は長さ 3 の one-hot ではなく **multi-hot**（例： [1,0,1]）。
#    後段の学習では BCEWithLogitsLoss（独立ベルヌーイ）を用いるのが標準。
#  - マルチクラス（排他的 1 ラベル）で扱いたい場合は、「優先順位ルール」や
#    「確率最大クラスを1つ選ぶ」などの単一化手続きを別途設ける。
# --------------------------------------------------------------------------------

import glob
import json
from collections import Counter

# 極性 → ラベルID の対応（ID順は学習・評価で再利用するため**固定**しておく）
category_id = {"negative": 0, "neutral": 1, "positive": 2}
id_to_category = {v: k for k, v in category_id.items()}  # デバッグ・可視化用の逆引き

dataset = []  # 出力：[{ 'text': <str>, 'labels': [neg, neu, pos] }, ...] の配列

# chABSA の JSON を列挙（ソートしておくと再現性が増す）
for file in sorted(glob.glob("chABSA-dataset/*.json")):
    # JSON をロード（UTF-8 を想定）
    with open(file, mode="r", encoding="utf-8") as f:
        data = json.load(f)

    # 各ファイルは {"sentences": [...]} の構造を想定
    # sentence 例：
    #   {
    #     "sentence": "・・・",
    #     "opinions": [
    #        {"polarity": "positive", ...},
    #        {"polarity": "negative", ...},
    #        ...
    #     ]
    #   }
    for sentence in data.get("sentences", []):
        text = sentence.get("sentence", "")

        # 文レベルのマルチラベルベクトル（[neg, neu, pos]）
        # - 同一文に複数 opinion がある場合、それぞれの極性に 1 を立てる（重複は 1 のまま）
        labels = [0, 0, 0]
        for opinion in sentence.get("opinions", []):
            pol = opinion.get("polarity")
            if pol in category_id:
                labels[category_id[pol]] = 1
            else:
                # 未知の極性が来た場合の保険（データ品質に依存）
                # 実務ではログ出力・スキップ数カウント等の監視を入れるとよい
                pass

        # 1 サンプル（テキスト + マルチラベル）を追加
        sample = {"text": text, "labels": labels}
        dataset.append(sample)

# ------------------（任意）簡易サニティチェックと統計------------------
# ラベル出現数（クラス別の陽性総数）を集計してみる
label_totals = Counter()
for s in dataset:
    for i, bit in enumerate(s["labels"]):
        if bit:
            label_totals[id_to_category[i]] += 1

print(f"# samples: {len(dataset)}")
print("# positives per class:", dict(label_totals))
print("# example:", dataset[0] if dataset else None)

# ここで得られた `dataset` は、後段の符号化フェーズで
# tokenizer(text, ...) → input_ids/attention_mask/... を作り、
# labels を torch.tensor([B, 3]) の float 型（{0.0,1.0}）にして BCEWithLogitsLoss へ渡す想定。

# samples: 6119
# positives per class: {'neutral': 230, 'positive': 2210, 'negative': 1746}
# example: {'text': '当連結会計年度におけるわが国経済は、政府の経済政策や日銀の金融緩和策により、企業業績、雇用・所得環境は改善し、景気も緩やかな回復基調のうちに推移いたしましたが、中国をはじめとするアジア新興国経済の減速懸念や、英国の欧州連合（ＥＵ）離脱決定、米国新政権への移行など、引き続き先行きは不透明な状況となっております', 'labels': [0, 1, 1]}


In [10]:
# 7-11
print(dataset[0])

{'text': '当連結会計年度におけるわが国経済は、政府の経済政策や日銀の金融緩和策により、企業業績、雇用・所得環境は改善し、景気も緩やかな回復基調のうちに推移いたしましたが、中国をはじめとするアジア新興国経済の減速懸念や、英国の欧州連合（ＥＵ）離脱決定、米国新政権への移行など、引き続き先行きは不透明な状況となっております', 'labels': [0, 1, 1]}


In [11]:
# 7-12
# =============================================================================
# 説明コメント付き：chABSA 由来データの符号化（固定長化）→ テンソル化 → 分割 → DataLoader 作成
# -----------------------------------------------------------------------------
# 目的：
#  - 文テキストをトークナイズし、BERT へ入力可能な辞書（input_ids/attention_mask/...）＋ labels を作る
#  - 学習/検証/テストに 60/20/20 で分割し、ミニバッチを供給する DataLoader を用意する
#
# 理論メモ：
#  - BERT 系の学習は「**固定長化**（max_length + truncation + padding='max_length'）」で
#    バッチ形状を揃えると、スループットと再現性が安定しやすい
#  - 本タスクは**マルチラベル**（multi-hot）想定：後段は BCEWithLogitsLoss（内部で sigmoid 合成）
#    → labels は float（{0.0, 1.0}）が望ましい
#  - 分割は単純ランダムだとクラス/マルチラベル分布が歪むことがある
#    → 研究では「層化（Stratified / Iterative Stratification）」や
#       「グループ分割（ファイル単位）」が推奨（ここでは最小構成としてランダム）
#  - 再現性が必要なら、事前に random.seed(SEED) を固定しておく
# =============================================================================

# トークナイザのロード（MODEL_NAME は 7-5 等で定義済み：語彙ID整合のため同一モデル名を使う）
tokenizer = BertJapaneseTokenizer.from_pretrained(MODEL_NAME)

# 各データの形式を整える（固定長化）
max_length = 128
dataset_for_loader = []
for sample in dataset:
    text = sample["text"]  # 文テキスト
    labels = sample["labels"]  # multi-hot（例：[neg, neu, pos]）

    # 1) トークナイズ：固定長化（長文は切り詰め、短文は PAD で埋める）
    encoding = tokenizer(
        text, max_length=max_length, padding="max_length", truncation=True
    )

    # 2) ラベル付与（BCEWithLogitsLoss を前提に float 化するのが望ましい）
    encoding["labels"] = labels

    # 3) テンソル化
    #  - tokenizer の戻り値（list[int]）→ torch.long（int64）へ
    #  - labels は明示的に float32（{0.0, 1.0}）へ（int でも後段で float 化されるが、ここで揃えると安全）
    encoding = {k: torch.tensor(v) for k, v in encoding.items() if k != "labels"}
    encoding["labels"] = torch.tensor(labels, dtype=torch.float32)

    dataset_for_loader.append(encoding)

# データセットの分割（60/20/20）
# - 単純ランダム：クラス不均衡やデータリークに注意（必要に応じて層化/グループ分割に置換）
random.shuffle(dataset_for_loader)
n = len(dataset_for_loader)
n_train = int(0.6 * n)
n_val = int(0.2 * n)
dataset_train = dataset_for_loader[:n_train]  # 学習データ
dataset_val = dataset_for_loader[n_train : n_train + n_val]  # 検証データ
dataset_test = dataset_for_loader[n_train + n_val :]  # テストデータ

# データローダを作成
# - 学習のみ shuffle=True（SGD のバイアス低減）
# - val/test は順序固定で OK（再現性・デバッグのため）
# - バッチサイズは VRAM/メモリと相談（val/test は no_grad 前提で大きく取りやすい）
dataloader_train = DataLoader(dataset_train, batch_size=32, shuffle=True)
dataloader_val = DataLoader(dataset_val, batch_size=256, shuffle=False)
dataloader_test = DataLoader(dataset_test, batch_size=256, shuffle=False)

# 補足：
# - DataLoader は辞書の各キーごとにテンソルをバッチ結合（collate_fn 既定）するため、
#   上記の dict 形式（input_ids/attention_mask/token_type_ids/labels）のままで学習に投入可能
# - BCEWithLogitsLoss の pos_weight（shape=[C]）で不均衡補正を入れると安定しやすい
# - グループ分割（例：source_file 単位）を採用するなら、7-10 で sample にメタを持たせるとよい

In [ ]:
# 7-13（修正＆説明コメント付き：PL 2.x の fit 引数名を正しく指定）
# =====================================================================
# エラー原因：
# - PyTorch Lightning 2.x の Trainer.fit は `dataloaders=` を受け付けず、
#   学習データは `train_dataloaders=...`、検証データは `val_dataloaders=...` を使う。
#   → `TypeError: Trainer.fit() got an unexpected keyword argument 'dataloaders'`
#     はこの非対応引数名が原因。
# 対処：
# - `trainer.fit(model, train_dataloaders=dataloader_train, val_dataloaders=dataloader_val)`
#   に修正する。
# ついでに：
# - Mac(MPS)/CUDA/CPU 自動選択（accelerator/devices）
# - ログの on_epoch 明示化
# - test_step で batch を破壊しないようにコピーして使用
# =====================================================================

import torch
import pytorch_lightning as pl

# --- 7-4で定義済み ---
# class BertForSequenceClassificationMultiLabel(torch.nn.Module): ...


class BertForSequenceClassificationMultiLabel_pl(pl.LightningModule):

    def __init__(self, model_name, num_labels, lr):
        """
        model_name: 事前学習BERT名（tokenizer と同一語彙を使うこと）
        num_labels: マルチラベルの次元数（例：3 → [neg, neu, pos]）
        lr       : 学習率（BERT微調整の一般的レンジは 1e-5〜5e-5）
        """
        super().__init__()
        self.save_hyperparameters()  # ハイパーパラメータを保存（再現性・ckpt再現に有用）

        # BERT本体 + mean-pooling + Linear → 各ラベルのロジット（活性前）
        self.bert_scml = BertForSequenceClassificationMultiLabel(
            model_name, num_labels=num_labels
        )

    def training_step(self, batch, batch_idx):
        # labels を含む dict をそのまま渡すと BCEWithLogitsLoss を内部計算（7-4のforward）
        output = self.bert_scml(**batch)
        loss = output.loss
        # エポック平均・プログレスバー表示を明示
        self.log("train_loss", loss, on_step=False, on_epoch=True, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        output = self.bert_scml(**batch)
        val_loss = output.loss
        self.log("val_loss", val_loss, on_step=False, on_epoch=True, prog_bar=True)

    def test_step(self, batch, batch_idx):
        # 破壊的に pop しない（後段のフックや再利用に安全）
        labels = batch["labels"]  # [B, C]（float 0/1 想定）
        inputs = {k: v for k, v in batch.items() if k != "labels"}
        output = self.bert_scml(**inputs)
        scores = output.logits  # [B, C] ロジット
        probs = scores.sigmoid()  # p(y_c=1|x)
        labels_pred = (probs >= 0.5).to(torch.int64)  # 閾値 τ=0.5
        labels_int = labels.to(torch.int64)
        # subset accuracy（全ラベル一致率：厳しめの指標）
        num_correct = (labels_pred == labels_int).all(dim=-1).sum().item()
        accuracy = num_correct / scores.size(0)
        self.log("accuracy", accuracy, on_step=False, on_epoch=True, prog_bar=True)

    def configure_optimizers(self):
        # 最小構成：Adam（実務は AdamW + weight_decay + scheduler 推奨）
        return torch.optim.Adam(self.parameters(), lr=self.hparams.lr)


# ============================== コールバック & Trainer ==============================

checkpoint = pl.callbacks.ModelCheckpoint(
    monitor="val_loss",
    mode="min",
    save_top_k=1,
    save_weights_only=True,  # 版差あり。必要なら state_dict で別保存も併用
    dirpath="model/",
    filename="epoch={epoch}-val_loss={val_loss:.4f}",
)

early_stopping = pl.callbacks.EarlyStopping(monitor="val_loss", mode="min", patience=2)


def pick_accelerator_and_devices():
    # Mac(Apple Silicon) 優先 → CUDA → CPU
    if torch.backends.mps.is_available() and torch.backends.mps.is_built():
        return "mps", 1
    if torch.cuda.is_available():
        return "gpu", 1
    return "cpu", 1


accelerator, devices = pick_accelerator_and_devices()

trainer = pl.Trainer(
    accelerator=accelerator,  # "mps"/"gpu"/"cpu"（または "auto" でも可）
    devices=devices,  # 単一デバイス
    max_epochs=5,
    callbacks=[checkpoint, early_stopping],
    # 任意：deterministic=True, gradient_clip_val=1.0, precision=16 など
)

# ============================== 学習・テスト実行 ==============================

model = BertForSequenceClassificationMultiLabel_pl(
    MODEL_NAME, num_labels=3, lr=1e-5  # [neg, neu, pos]
)

# 【FIX】PL 2.x の正しい引数名：train_dataloaders / val_dataloaders
trainer.fit(model, train_dataloaders=dataloader_train, val_dataloaders=dataloader_val)

# テストは dataloaders=... でOK（PL 2.x 仕様）
test = trainer.test(model, dataloaders=dataloader_test, ckpt_path="best")
print(f'Accuracy: {test[0]["accuracy"]:.2f}')

In [ ]:
# 7-14（説明コメント付き：学習済みマルチラベルBERTで新規テキストを推論）
# =============================================================================
# 目的：
#  - 7-13 の学習で保存された「最良 ckpt」をロードし、任意の文章に対して
#    マルチラベル（例： [neg, neu, pos] ）の予測を行う。
#
# 理論メモ：
#  - 本モデルは logits（活性前スコア）を出力する。確率化には sigmoid を用い、
#    しきい値 τ（既定 0.5）で各ラベルの 0/1 を決める（独立ベルヌーイ仮定）。
#  - 推論時は Dropout を停止するため eval()、かつ勾配不要の no_grad() を使う。
#  - デバイスは MPS（Mac）→ CUDA → CPU の順で自動選択し、入出力テンソルとモデルを揃える。
#  - トークナイズは本来、学習時と同じ固定長化（max_length + truncation + padding='max_length'）
#    が望ましいが、ここでは可読性を優先して padding='longest' を利用（小規模推論向け）。
# =============================================================================

import os
import glob
import torch
from transformers import BertJapaneseTokenizer

# --- ラベルIDの定義（7-10 と整合） ---
category_id = {"negative": 0, "neutral": 1, "positive": 2}
id_to_category = {v: k for k, v in category_id.items()}

# --- 入力する文章 ---
text_list = [
    "今期は売り上げが順調に推移したが、株価は低迷の一途を辿っている。",
    "昨年から黒字が減少した。",
    "今日の飲み会は楽しかった。",
]


# --- デバイス選択：Mac(MPS) → CUDA → CPU の順 ---
def get_best_device() -> torch.device:
    if torch.backends.mps.is_available() and torch.backends.mps.is_built():
        return torch.device("mps")
    if torch.cuda.is_available():
        return torch.device("cuda")
    return torch.device("cpu")


device = get_best_device()
print(f"[info] device = {device}")

# --- トークナイザのロード（学習時と同一 MODEL_NAME を使用） ---
try:
    MODEL_NAME
except NameError:
    MODEL_NAME = "tohoku-nlp/bert-base-japanese-whole-word-masking"
tokenizer = BertJapaneseTokenizer.from_pretrained(MODEL_NAME)


# --- 最良 ckpt のパスを取得 ---
# 7-13 の checkpoint.best_model_path が生きていればそれを使い、
# 無ければ 'model/' ディレクトリから新しい .ckpt を探索して拾う。
def resolve_best_ckpt():
    # 1) checkpoint.best_model_path がグローバルに存在する場合
    if "checkpoint" in globals():
        try:
            p = checkpoint.best_model_path
            if isinstance(p, str) and os.path.exists(p):
                return p
        except Exception:
            pass
    # 2) フォールバック：model/ 以下の .ckpt を更新時刻順で最後の1件
    cks = sorted(glob.glob("model/*.ckpt"), key=os.path.getmtime)
    if not cks:
        raise FileNotFoundError(
            "最良 ckpt が見つかりません（model/*.ckpt が存在しません）。"
        )
    return cks[-1]


best_model_path = resolve_best_ckpt()
print(f"[info] best ckpt = {best_model_path}")

# --- LightningModule を ckpt から復元し、中の bert_scml（純PyTorchモジュール）を取り出す ---
# 7-13 の LightningModule は self.save_hyperparameters() 済みのため、追加引数なしで復元可。
model = BertForSequenceClassificationMultiLabel_pl.load_from_checkpoint(best_model_path)
bert_scml = model.bert_scml.to(device).eval()  # 推論モードに切替

# --- データの符号化 ---
# 小規模推論のため padding='longest' を使う（大量推論は固定長化の方が安定）。
encoding = tokenizer(text_list, padding="longest", return_tensors="pt")
encoding = {k: v.to(device) for k, v in encoding.items()}

# --- 予測（logits → sigmoid → 閾値判定） ---
with torch.no_grad():
    output = bert_scml(**encoding)
scores = output.logits  # [B, C]（活性前）
probs = scores.sigmoid()  # [B, C]（各ラベルの確率）
threshold = 0.5
labels_predicted = (probs >= threshold).to(torch.int64).cpu().tolist()

# --- 結果を表示（ラベル名に変換） ---
print("\n# 推論結果")
for text, label_vec in zip(text_list, labels_predicted):
    active = [id_to_category[i] for i, v in enumerate(label_vec) if v == 1]
    print("--")
    print(f"入力：{text}")
    print(f"出力（multi-hot） ：{label_vec}")
    print(f"出力（ラベル名） ：{active}")

# =============================================================================
# 補足（実務向けの推奨）：
#  - 学習と推論でトークナイズ設定（max_length など）を揃える。
#  - 閾値 0.5 は便宜的。検証セットで PR/ROC に基づき τ を最適化するか、
#    ラベル別に τ_c を持たせると性能が上がりやすい。
#  - クラス不均衡が強い場合、学習時に BCEWithLogitsLoss(pos_weight=...) を導入。
#  - しきい値最適化後は、micro/macro-F1・Hamming loss などを併記すると挙動把握が容易。
# =============================================================================